### Homework Week 2

In [12]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

In [5]:
df1 = pd.read_csv('homework_2.1.csv', index_col=0, skipinitialspace=True)
df2 = pd.read_csv('homework_2.2.csv', index_col=0, skipinitialspace=True)

## Question 1

In [13]:
df_long = df1.melt(id_vars="time", value_vars=["G1", "G2", "G3"],
                   var_name="group", value_name="outcome")
model = smf.ols("outcome ~ time + C(group)", data=df_long).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                outcome   R-squared:                       0.311
Model:                            OLS   Adj. R-squared:                  0.304
Method:                 Least Squares   F-statistic:                     44.55
Date:                Sun, 21 Sep 2025   Prob (F-statistic):           8.72e-24
Time:                        13:06:52   Log-Likelihood:                -216.89
No. Observations:                 300   AIC:                             441.8
Df Residuals:                     296   BIC:                             456.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.0786      0.071      1.

In [15]:

# Ensure G1 is the reference category
df_long["group"] = pd.Categorical(df_long["group"], categories=["G1","G2","G3"])

# Fit: shared slope + group fixed effects
model = smf.ols("outcome ~ time + C(group)", data=df_long).fit()

# Coefficient for Group 1 (its intercept)
coef_G1_intercept = model.params["Intercept"]

# Slope shared by all groups
shared_slope = model.params["time"]

coef_G1_intercept, shared_slope


(np.float64(0.07855194563250359), np.float64(0.009017213376458716))

In [16]:
import pandas as pd
import statsmodels.formula.api as smf

# Load and reshape
df = pd.read_csv("homework_2.1.csv")
df_long = df.melt(id_vars="time", value_vars=["G1","G2","G3"],
                  var_name="group", value_name="outcome")

# Ensure G1 is the reference category
df_long["group"] = pd.Categorical(df_long["group"], categories=["G1","G2","G3"])

# Fit: shared slope + group fixed effects
model = smf.ols("outcome ~ time + C(group)", data=df_long).fit()

# Coefficient for Group 1 (its intercept)
coef_G1_intercept = model.params["Intercept"]

# Slope shared by all groups
shared_slope = model.params["time"]

coef_G1_intercept, shared_slope

(np.float64(0.07855194563250359), np.float64(0.009017213376458716))

In [19]:
model_int = smf.ols("outcome ~ time * C(group)", data=df_long).fit()

params = model_int.params

alpha_G1 = params["Intercept"]
beta_G1  = params["time"]

alpha_G2 = alpha_G1 + params["C(group)[T.G2]"]
beta_G2  = beta_G1 + params["time:C(group)[T.G2]"]

alpha_G3 = alpha_G1 + params["C(group)[T.G3]"]
beta_G3  = beta_G1 + params["time:C(group)[T.G3]"]

## Question 2

In [20]:
df_long = df1.melt(id_vars="time", value_vars=["G1", "G2", "G3"],
                   var_name="group", value_name="outcome")
model = smf.ols("outcome ~ time + C(group)", data=df_long).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                outcome   R-squared:                       0.311
Model:                            OLS   Adj. R-squared:                  0.304
Method:                 Least Squares   F-statistic:                     44.55
Date:                Sun, 21 Sep 2025   Prob (F-statistic):           8.72e-24
Time:                        13:25:41   Log-Likelihood:                -216.89
No. Observations:                 300   AIC:                             441.8
Df Residuals:                     296   BIC:                             456.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.0786      0.071      1.

## Question 3

In [27]:
import pandas as pd

# Suppose Y is the outcome variable
treated_mean = df2.loc[df2['X'] == 1, 'Y'].mean()
untreated_mean = df2.loc[df2['X'] == 0, 'Y'].mean()

naive_effect_Y = treated_mean - untreated_mean

print("Naïve effect of treatment on Y:", naive_effect_Y)

# If Z is the outcome instead:
treated_mean_Z = df2.loc[df2['X'] == 1, 'Z'].mean()
untreated_mean_Z = df2.loc[df2['X'] == 0, 'Z'].mean()

naive_effect_Z = treated_mean_Z - untreated_mean_Z

print("Naïve effect of treatment on Z:", naive_effect_Z)

Naïve effect of treatment on Y: 2.92071726472051
Naïve effect of treatment on Z: 0.04801609188227575


## Question 4

In [29]:
def naive_effect(df, outcome_col, treat_col="X"):
    treated = df.loc[df[treat_col] == 1, outcome_col]
    control = df.loc[df[treat_col] == 0, outcome_col]
    return treated.mean() - control.mean()

def bootstrap_variance_naive_effect(
    df, outcome_col, treat_col="X", n_boot=10000, random_state=42
):
    rng = np.random.default_rng(random_state)
    n = len(df)
    idx = np.arange(n)
    effects = np.empty(n_boot, dtype=float)
    for b in range(n_boot):
        sample_idx = rng.choice(idx, size=n, replace=True)
        effects[b] = naive_effect(df.iloc[sample_idx], outcome_col, treat_col)
    return {
        "point_estimate": naive_effect(df, outcome_col, treat_col),
        "bootstrap_variance": effects.var(ddof=1),
        "bootstrap_se": effects.std(ddof=1),
        "effects": effects,  # keep if you want CI's
    }

# Example: use Y as the outcome
res_Y = bootstrap_variance_naive_effect(df2, outcome_col="Y", treat_col="X", n_boot=100000, random_state=42)
print("Naïve effect on Y:", res_Y["point_estimate"])
print("Bootstrap variance (Y):", res_Y["bootstrap_variance"])
print("Bootstrap SE (Y):", res_Y["bootstrap_se"])

# If Z is the outcome instead:
res_Z = bootstrap_variance_naive_effect(df2, outcome_col="Z", treat_col="X", n_boot=100000, random_state=42)
print("Naïve effect on Z:", res_Z["point_estimate"])
print("Bootstrap variance (Z):", res_Z["bootstrap_variance"])
print("Bootstrap SE (Z):", res_Z["bootstrap_se"])

Naïve effect on Y: 2.92071726472051
Bootstrap variance (Y): 0.031615455191033315
Bootstrap SE (Y): 0.177807354153402
Naïve effect on Z: 0.04801609188227575
Bootstrap variance (Z): 0.0006031058226652361
Bootstrap SE (Z): 0.024558212937126272


## Question 5

In [31]:
import numpy as np
import pandas as pd
from scipy.stats import skew

def ols_coef_treat(y, x):
    # Design matrix with intercept and treatment
    X = np.column_stack([np.ones(len(x), dtype=float), x.astype(float)])
    # OLS closed-form: beta = (X'X)^(-1) X'y
    beta = np.linalg.pinv(X.T @ X) @ (X.T @ y.astype(float))
    return beta[1]  # treatment coefficient

def bootstrap_effect_skewness(df, outcome_col="Y", treat_col="X", n_boot=100000, random_state=42):
    rng = np.random.default_rng(random_state)
    n = len(df)
    idx = np.arange(n)
    coefs = np.empty(n_boot, dtype=float)
    for b in range(n_boot):
        sample_idx = rng.choice(idx, size=n, replace=True)
        d = df.iloc[sample_idx]
        coefs[b] = ols_coef_treat(d[outcome_col].values, d[treat_col].values)
    return {
        "point_estimate": ols_coef_treat(df[outcome_col].values, df[treat_col].values),
        "skewness": skew(coefs, bias=False, nan_policy="omit"),
        "bootstrap_se": coefs.std(ddof=1),
        "coefs": coefs,  # keep if you want to inspect or build CIs
    }

# Run for Y as the outcome (change outcome_col="Z" to use Z instead)
res = bootstrap_effect_skewness(df2, outcome_col="Y", treat_col="X", n_boot=100000, random_state=42)
print("OLS treatment effect (Y):", res["point_estimate"])
print("Bootstrap SE:", res["bootstrap_se"])
print("Skewness of bootstrapped effect:", res["skewness"])

OLS treatment effect (Y): 2.9207172647204978
Bootstrap SE: 0.1778073541534017
Skewness of bootstrapped effect: 0.049015196938418305
